In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#---------------------------------------------------
#
# Config *** TO UPDATE ***
#
#---------------------------------------------------

# *** TO UPDATE: change the team number with your own team number
TEAM_NUMBER = "Example 1"

# File with AOIS definitions 
AOIS_DEFINITION_FILE = "../raw-data/"+TEAM_NUMBER+"/aois definitions/aois.csv"

# This file should be automatically generated from "4. Mapping Fixations and Saccades to AOIs"
FIFXATIONS_SACCADES_WITH_AOIS_FILE = "data/fixationAndSaccadesDataWithAOIs.csv"

# This file should be automatically generated from "5. AOI Visits"
DWELLS_FILE = "data/dwells.csv"

In [ ]:
# Read aois definition
aois_df = pd.read_csv(AOIS_DEFINITION_FILE)

In [ ]:
# Display aois definition
display(aois_df)

In [ ]:
# get list of AOIs 
aois = aois_df["AOI"].tolist()
print(f'AOIs: {aois}')

In [ ]:
# Read fixation and saccades data with AOIs using pandas library
fixationAndSaccadeData = pd.read_csv(FIFXATIONS_SACCADES_WITH_AOIS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# Preview fixationAndSaccadeData
fixationAndSaccadeData.head()

In [ ]:
#----------------------------------------------------------------------------------------
#
# 1. Fixation measures at stimulus and AOI levels
#
#----------------------------------------------------------------------------------------

In [ ]:
#filter out data where FixID is NaN
fixationData = fixationAndSaccadeData[fixationAndSaccadeData['FixID'].notnull()].copy(deep=True)

In [ ]:
# Preview fixationData
fixationData.head()

In [ ]:
allAOIFixationStats = []

# Metrics for each task at the level of the whole stimulus and individual AOIs
for task in fixationData['SourceStimuliName'].unique():
    taskFixationData = fixationData[fixationData['SourceStimuliName'] == task]
    
    # Get task start times per participant
    task_start_times = taskFixationData.groupby('Respondent')['Fixation Start'].min()
    
    # Create a list of AOIs including 'Whole Stimulus'
    aois_to_process = ['Whole Stimulus'] + aois
    
    # Iterate through the AOIs, including 'Whole Stimulus'
    for aoi in aois_to_process:

        if aoi == 'Whole Stimulus':
            # For the whole stimulus, use all data for the task
            aoiData = taskFixationData.copy()
        else:
            # Filter data where the AOI is fixated (AOI value is 1)
            aoiData = taskFixationData[taskFixationData[aoi] == 1]
        
        if not aoiData.empty:
            # Compute metrics per participant
            participant_metrics = aoiData.groupby('Respondent').agg(
                Number_of_Fixations=('FixID', 'count'),
                Total_Fixation_Duration=('Fixation Duration', 'sum'),
                Average_Fixation_Duration=('Fixation Duration', 'mean'),
                First_Fixation_Time=('Fixation Start', 'min')
            ).reset_index()

            
            # Add task start times
            participant_metrics = participant_metrics.merge(
                task_start_times.rename('Task_Start_Time').reset_index(),
                on='Respondent'
            )
            
            # Compute Time to First Fixation
            participant_metrics['Time_to_First_Fixation'] = (
                participant_metrics['First_Fixation_Time'] - participant_metrics['Task_Start_Time']
            )
            
            # For 'Whole Stimulus', no value for Time to First Fixation
            if aoi == 'Whole Stimulus':
                participant_metrics['Time_to_First_Fixation'] = np.nan
            
            # Compute the average of these metrics across participants
            numeric_columns = ['Number_of_Fixations', 'Total_Fixation_Duration', 'Average_Fixation_Duration', 'Time_to_First_Fixation']
            avg_metrics = participant_metrics[numeric_columns].mean()
            
            stats = {
                'Task': task,
                'AOI': aoi,
                'Number of Fixations': avg_metrics['Number_of_Fixations'],
                'Total Fixation Duration': avg_metrics['Total_Fixation_Duration'],
                'Average Fixation Duration': avg_metrics['Average_Fixation_Duration'],
                'Time to First Fixation': avg_metrics['Time_to_First_Fixation']
            }
            
            allAOIFixationStats.append(stats)

# Convert the list of dictionaries to a DataFrame
allAOIFixationStats = pd.DataFrame(allAOIFixationStats)


In [ ]:
# Display the metrics for all AOIs aggregated
print("Note: Metrics averaged per participant")
display(allAOIFixationStats)

In [ ]:
# export allAOIFixationStats to csv 
allAOIFixationStats.to_csv("data/aoiFixationStats.csv", index=False)

In [ ]:
#----------------------------------------------------------------------------------------
#
# 2. Saccade measures at stimulus and AOI levels 
#
#----------------------------------------------------------------------------------------

In [ ]:
#filter out data where SacID is NaN
SaccadeData = fixationAndSaccadeData[fixationAndSaccadeData['SacID'].notnull()].copy(deep=True)

In [ ]:
# Preview Saccade data
SaccadeData.head()

In [ ]:
allAOISaccadeStats = []

# Metrics for each task at the level of the whole stimulus and individual AOIs
for task in SaccadeData['SourceStimuliName'].unique():
    taskSaccadeData = SaccadeData[SaccadeData['SourceStimuliName'] == task]
    
    # Create a list of AOIs including 'Whole Stimulus'
    aois_to_process = ['Whole Stimulus'] + aois
    
    # Iterate through the AOIs, including 'Whole Stimulus'
    for aoi in aois_to_process:
        if aoi == 'Whole Stimulus':
            # For the whole stimulus, use all data for the task
            aoiData = taskSaccadeData.copy()
        else:
            # Filter data where the AOI is fixated (AOI value is 1)
            aoiData = taskSaccadeData[taskSaccadeData[aoi] == 1]
        
        if not aoiData.empty:
            # Compute metrics per participant
            participant_metrics = aoiData.groupby('Respondent').agg(
                Number_of_Saccades=('SacID', 'count'),
                Total_Saccade_Duration=('Saccade Duration', 'sum'),
                Average_Saccade_Duration=('Saccade Duration', 'mean'),
                Total_Saccade_Amplitude=('Saccade Amplitude', 'sum'),
                Average_Saccade_Amplitude=('Saccade Amplitude', 'mean'),
                Average_Peak_Velocity_of_Saccades=('Saccade Peak Velocity', 'mean')
            ).reset_index()
            
            # Compute the average of these metrics across participants
            numeric_columns = [
                'Number_of_Saccades',
                'Total_Saccade_Duration',
                'Average_Saccade_Duration',
                'Total_Saccade_Amplitude',
                'Average_Saccade_Amplitude',
                'Average_Peak_Velocity_of_Saccades'
            ]
            avg_metrics = participant_metrics[numeric_columns].mean()
            
            stats = {
                'Task': task,
                'AOI': aoi,
                'Number of Saccades': avg_metrics['Number_of_Saccades'],
                'Total Saccade Duration': avg_metrics['Total_Saccade_Duration'],
                'Average Saccade Duration': avg_metrics['Average_Saccade_Duration'],
                'Total Saccade Amplitude': avg_metrics['Total_Saccade_Amplitude'],
                'Average Saccade Amplitude': avg_metrics['Average_Saccade_Amplitude'],
                'Average Peak Velocity of Saccades': avg_metrics['Average_Peak_Velocity_of_Saccades']
            }
            
            allAOISaccadeStats.append(stats)

# Convert the list of dictionaries to a DataFrame
allAOISaccadeStats = pd.DataFrame(allAOISaccadeStats)


In [ ]:
# Display the metrics for all AOIs aggregated
print("Note: Metrics averaged per participant")
display(allAOISaccadeStats)

In [ ]:
# export allAOISaccadeStats to csv 
allAOISaccadeStats.to_csv("data/aoiSaccadeStats.csv", index=False)

In [ ]:
#----------------------------------------------------------------------------------------
#
# 3. Dwell measures 
#
#----------------------------------------------------------------------------------------

In [ ]:
# Read dwell data using pandas library
dwells = pd.read_csv(DWELLS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# Preview dwells
dwells.head()

In [ ]:
dwellsStats = []

# Compute metrics per participant, VisitedAOI, and SourceStimuliName
participant_metrics = dwells.groupby(['VisitedAOI', 'SourceStimuliName', 'Respondent']).agg(
    Total_Dwell_Time=('Dwell Time', 'sum'),
    Average_Dwell_Time=('Dwell Time', 'mean'),
    Number_of_Dwells=('Dwell Time', 'count'),
    Average_Number_of_Fixations_in_Dwell=('Number of Fixations in Dwell', 'mean'),
    Average_Number_of_Saccades_in_Dwell=('Number of Saccades in Dwell', 'mean')
).reset_index()

# Exclude 'Respondent' before computing the mean
numeric_columns = participant_metrics.select_dtypes(include='number').columns

# Now compute the average of these metrics across participants
grouped_metrics = participant_metrics.groupby(['VisitedAOI', 'SourceStimuliName'])[numeric_columns].mean().reset_index()

# Build the stats dictionary
for _, row in grouped_metrics.iterrows():
    stats = {
        'VisitedAOI': row['VisitedAOI'],
        'SourceStimuliName': row['SourceStimuliName'],
        'Total Dwell Time': row['Total_Dwell_Time'],
        'Average Dwell Time': row['Average_Dwell_Time'],
        'Number of Dwells': row['Number_of_Dwells'],
        'Average Number of Fixations in Dwell': row['Average_Number_of_Fixations_in_Dwell'],
        'Average Number of Saccades in Dwell': row['Average_Number_of_Saccades_in_Dwell']
    }
    dwellsStats.append(stats)

# Convert the list of dictionaries to a DataFrame
dwellsStats = pd.DataFrame(dwellsStats)


In [ ]:
# Display dwellsStats
print("Note: Metrics averaged per participant")
display(dwellsStats)

In [ ]:
# Export dwellsStats to csv 
dwellsStats.to_csv("data/dwellsStats.csv", index=False)